## Motivating Example

You are in a room of n people (including yourself). Each person in the room has contributed \$1 to a central pot, so there is a total of $n in the pot. The money in the pot will be redistributed back to the people in the room, in the following way: each dollar is equally likely to go to any one of the n people, independently of the other dollars in the pot. This means that some people could get more than $1, while others end up with nothing.

As $n \rightarrow \inf$, what is the probability that you end up with no money? There are two common schools of thought:
- As $n \rightarrow \inf$, the number of dollars in the pot increases to infinity, so it seems that the probability that you end up with at least one of those dollars should approach 1, i.e., the probability that you end up with no money approaches 0.
- As $n \rightarrow \inf$, the chance that you earn each dollar, $\frac{1}{n}$, decreases to 0, so it seems that the probability that you end up with no money approaches 1.

## Theory

### Solving the example

- Let's model this as a draw with replacement
    - There are $n$ people in the room 
    - At each turn, we draw one of $n$ names with replacement to decide who gets the $1
    - So the probability that you end up with no money for the entire draw is is:
        $$f(0) = \binom{n}{0} \frac{1}{n}^0 (1-\frac{1}{n})^n = (1-\frac{1}{n})^n$$

- Using the known mathematical expression:
    - $$\begin{align}
        \lim_{n \rightarrow \inf} log(1 - \frac{1}{n})^n &= \lim_{n \rightarrow \inf} n log(1 - \frac{1}{n}) \\
        &= \lim_{x=\frac{1}{n} \rightarrow 0} \frac{log(1 - x)}{x} & (\text{Let x be 1/n}) \\ 
        &= \lim_{x=\frac{1}{n} \rightarrow 0} \frac{\frac{\partial }{\partial x}log(1 - x)}{\frac{\partial }{\partial x}x} & (\text{L'Hopital's rule})\\  
        &= \lim_{x=\frac{1}{n} \rightarrow 0} \frac{-\frac{1}{1-x}}{1} \\ 
        &= -1
        \end{align}$$

- Now we know that limit of $log(1 - \frac{1}{n})^n$ is -1 as n approaches infinity
    - Hence, $\lim_{n \rightarrow \inf} (1 - \frac{1}{n})^n = e^{-1}$

- So the probability that you get no money approaches $\frac{1}{e}$ as the number of people reaches $\inf$

### Generalising the observation above

- For **any** binomial with large $n$ and small $p$, you can approximate it with a PMF involving $e$ in some way!

- Theorem 16.1: Let $X\sim\text{Binomial}(n, p=\frac{\mu}{n})$, where $\mu$ is constant. Then as $n \rightarrow \inf$, the PMF of X approaches

$$
    f(x) = e^{-\mu} \frac{\mu^x}{x!}
$$
- A variable with this PMF is said to follow a poisson distribution!!

- Example 16.1 Motivating example
    - In the motivating example, notice that $\mu = 1$ (because you have a 1/n chance of getting a dollar for each draw)
    - As such, $f(x) = e^{-1} \frac{1^{x}}{x!}$
    - You are interested to know the probability of NOT getting a dollar
    - So $f(0) = e^{-1} \frac{1^{0}}{0!} = e^{-1}$, which matches whatever we found earlier


- Example 16.2 
    - The number of typos in a New York Times op-ed when it reaches the copy editor is a $\text{Poisson}(\mu=4.6)$ random variable. What is the probability that there are 2 or more typos in a randomly selected op-ed?
        - Let X be the number of typos in the op-ed
        - We want to know P(X >= 2)
        - This is 1 - P(X <= 1) = 1 - F(1) = 1 - f(1) - f(0)
    - Plugging into the formula:
        - $f(1) = e^{-4.6} \frac{4.6^{1}}{1!} = 0.046$
        - $f(0) = e^{-4.6} \frac{4.6^{0}}{0!} = 0.010$
    - $P(X >= 2) = 1 - 0.046 - 0.010 = 0.944$

In [10]:
import numpy as np
print(1 - np.exp(-4.6) * 4.6 - np.exp(-4.6))

import scipy
1 - scipy.stats.poisson.cdf(1, mu=4.6)

0.9437097198300519


0.9437097198300519

- More broadly, why is example 16.2 a good use of the poisson distribution?
    - Let's suppose the rate at which typos occur is binomial
    - There are many words, so $n$ is large
    - The probability of typos is probably small, so $p$ is small
    - Hence, using a Poisson here works

- Example 16.3:
    - A data center has 10,000 disk drives. Suppose that a disk drive fails in a given day with probability $10^{−3}$
        - What is the probability that there are 12 or more disk failures tomorrow?
            - Let X be number of disk failures
            - Using binomial
                - X~Binomial(10_000, 10^-3)
                - P(X >= 12) = 1 - P(X <= 11) = 1 - F(11) = 1 - f(11) - f(10) - ...
            - Using poisson
                - X~Poisson(10^-3 * 10_000)
                - P(X >= 12) = 1 - P(X <= 11) = 1 - f(11) - f(10) - ...
                

In [24]:
import scipy

print(1 - np.sum(scipy.stats.binom.pmf(n=10_000, p=10**-3, k=range(12))))

print(1 - np.sum(scipy.stats.poisson.pmf(mu=10**-3 * 10_000, k=range(12))))

0.30316693332507105
0.303223853696892


- How many spare disk drives should be available so that all failures in a day can be replaced with probability 99%?
    - In the previous part, we found that P(X >= 12) is 30.3%
    - We want to know some value $b$ such that P(X >= b) is 1%
    - That is, there is 1% or less chance that more than $b$ drives will fail in a day, so by keeping $b$ drivers as spares, we mitigate any problems
    - Following the earlier approach, the earliest value of $b$ where $P(X >= b) > 0.99$ is 18

In [34]:
display([(x-1, np.sum(scipy.stats.binom.pmf(n=10_000, p=10**-3, k=range(x)))) for x in range(12, 20)])
[(x-1, np.sum(scipy.stats.poisson.pmf(mu=10_000*10**-3, k=range(x)))) for x in range(12, 20)]

[(11, 0.696833066674929),
 (12, 0.7916513294452172),
 (13, 0.8645738526260627),
 (14, 0.9166457272869453),
 (15, 0.9513464106612242),
 (16, 0.9730234829517759),
 (17, 0.9857670435564315),
 (18, 0.992841838657993)]

[(11, 0.696776146303108),
 (12, 0.791556476394876),
 (13, 0.8644644226193131),
 (14, 0.9165415270653393),
 (15, 0.9512595966960234),
 (16, 0.972958390215201),
 (17, 0.9857223864029525),
 (18, 0.9928134953961478)]

### Proof of theorem 16.1

- Let's start from the binomial pmf, and work our way to the poisson

$$\begin{align}
    f(x) &= \binom{n}{x} (\frac{\mu}{n}^x) (1 - \frac{\mu}{n})^{n-x} \\
    &= \frac{n!}{x! (n-x)!} \frac{\mu^x}{n^x} (1 - \frac{\mu}{n})^{n-x} \\
    &= \frac{n!}{n^{x} (n-x)!} \frac{\mu^x}{x!} (1 - \frac{\mu}{n})^{n-x} \\
    &\rightarrow 1 \cdot \frac{\mu^x}{x!} \cdot e^{-\mu}
\end{align}$$

### Visualising poisson PMF

- What happens to PMF when $\mu$ increases?
    - Shifts right

In [12]:
import scipy
display(scipy.stats.poisson.pmf(mu=3, k=range(10)))
display(scipy.stats.poisson.pmf(mu=7, k=range(10)))
display(scipy.stats.poisson.pmf(mu=11, k=range(10)))

array([0.04978707, 0.14936121, 0.22404181, 0.22404181, 0.16803136,
       0.10081881, 0.05040941, 0.02160403, 0.00810151, 0.0027005 ])

array([0.00091188, 0.00638317, 0.02234111, 0.05212925, 0.09122619,
       0.12771667, 0.14900278, 0.14900278, 0.13037743, 0.10140467])

array([1.67017008e-05, 1.83718709e-04, 1.01045290e-03, 3.70499396e-03,
       1.01887334e-02, 2.24152134e-02, 4.10945580e-02, 6.45771626e-02,
       8.87935985e-02, 1.08525509e-01])